# MArch MAdness Models, 2024

Data From Kaggle Competition Files

Year two of trying this. Going to use the actual evaluation Functions intead of just beating my frined's picks

### PAckages

In [64]:
import os
import pandas as pd
import numpy as np
import sklearn
import random

In [5]:
os.chdir('C:\\Users\\Nick\\Python Projects\\2024 March Madness\\MMML')

### Data Import

In [51]:
# I want Mteams, MRegularSeasonDetailedResults, MTeamConferences, MTourneyCompactResults, MTeamConferences
# MMasseyOrdinals, MTeamSpellings, 2024_tourney_seeds (not updated id bet)

MTeams = pd.read_csv('Data\\MTeams.csv')
WTeams = pd.read_csv('Data\\WTeams.csv')

MRegSeason = pd.read_csv('Data\\MRegularSeasonDetailedResults.csv')
WRegSeason = pd.read_csv('Data\\WRegularSeasonDetailedResults.csv')

MConf = pd.read_csv('Data\\MTeamConferences.csv')
WConf = pd.read_csv('Data\\WTeamConferences.csv')

MTourneyResults = pd.read_csv('Data\\MNCAATourneyCompactResults.csv')
WTourneyResults = pd.read_csv('Data\\WNCAATourneyCompactResults.csv')

MMassey = pd.read_csv('Data\\MMasseyOrdinals.csv')

MConfTOurney = pd.read_csv('Data\\MConferenceTourneyGames.csv')


In [52]:
RegSeason = pd.concat([MRegSeason, WRegSeason], ignore_index = True)
RegSeason.head(10)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
5,2003,11,1458,81,1186,55,H,0,26,57,...,11,12,17,6,22,8,19,4,3,25
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1186,75,1457,61,N,0,28,62,...,17,17,23,8,25,10,15,14,8,18
8,2003,12,1194,71,1156,66,N,0,28,58,...,18,12,27,13,26,13,25,8,2,18
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


In [53]:

SeasonWins = RegSeason.groupby(['WTeamID', 'Season'])['LTeamID'].count().reset_index().rename(columns = {'LTeamID':'Wins', 'WTeamID':'TeamID'})
SeasonLs = RegSeason.groupby(['LTeamID', 'Season'])['WTeamID'].count().reset_index().rename(columns = {'WTeamID':'Losses', 'LTeamID':'TeamID'})
OpponentsW = RegSeason.groupby(['WTeamID', 'Season'])['LTeamID'].apply(list).reset_index().rename(columns = {'LTeamID':'OpponentsBeaten', 'WTeamID':'TeamID'})
OpponentsL = RegSeason.groupby(['LTeamID', 'Season'])['WTeamID'].apply(list).reset_index().rename(columns = {'WTeamID':'OpponentsLost', 'LTeamID':'TeamID'})

#Merge these Columns
merged_df = pd.merge(SeasonWins, SeasonLs, on=['TeamID', 'Season'], how='inner')

merged_df1 = pd.merge(merged_df, OpponentsW, on=['TeamID', 'Season'], how='inner')

TeamRecords = pd.merge(merged_df1, OpponentsL, on=['TeamID', 'Season'], how='inner')

SeasonStats = TeamRecords

SeasonStats['Win_PCT'] = SeasonStats['Wins']/(SeasonStats['Wins'] + SeasonStats['Losses'])
TeamRecords.loc[TeamRecords.TeamID == 1361]

,TeamID,Season,Wins,Losses,OpponentsBeaten,OpponentsLost,Win_PCT
5363,1361,2003,14,13,"[1142, 1253, 1218, 1222, 1194, 1236, 1258, 136...","[1186, 1339, 1112, 1403, 1140, 1307, 1161, 142...",0.518519
5364,1361,2004,12,16,"[1253, 1326, 1360, 1142, 1186, 1339, 1235, 114...","[1173, 1407, 1403, 1112, 1449, 1388, 1161, 146...",0.428571
5365,1361,2005,9,18,"[1284, 1360, 1364, 1168, 1140, 1424, 1161, 114...","[1364, 1449, 1143, 1388, 1258, 1235, 1344, 146...",0.333333
5366,1361,2006,22,8,"[1379, 1107, 1364, 1420, 1258, 1161, 1428, 142...","[1227, 1360, 1143, 1450, 1344, 1102, 1428, 1140]",0.733333
5367,1361,2007,19,10,"[1293, 1388, 1143, 1415, 1364, 1360, 1144, 125...","[1444, 1112, 1450, 1461, 1140, 1102, 1424, 146...",0.655172
5368,1361,2008,19,12,"[1251, 1339, 1201, 1360, 1258, 1444, 1415, 129...","[1143, 1388, 1112, 1294, 1140, 1424, 1395, 130...",0.612903
5369,1361,2009,22,9,"[1201, 1441, 1370, 1214, 1294, 1360, 1364, 116...","[1113, 1112, 1388, 1461, 1140, 1428, 1307, 114...",0.709677
5370,1361,2010,23,8,"[1365, 1201, 1319, 1360, 1364, 1168, 1112, 117...","[1388, 1334, 1113, 1461, 1424, 1140, 1307, 1140]",0.741935
5371,1361,2011,30,2,"[1253, 1211, 1453, 1237, 1275, 1388, 1455, 114...","[1140, 1140]",0.937500
5372,1361,2012,24,7,"[1136, 1381, 1413, 1425, 1253, 1112, 1364, 114...","[1124, 1166, 1161, 1424, 1307, 1102, 1307]",0.774194


In [54]:
MRegSeason.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

Season stats need to be by team so I cant just group by Wteam because then I'd only have their stats from games they won. 
I also can't have Wteam columns in the game and tourney games data when I  train/make predictions. I need to somehow randomize this so sometimes Wteam is T1 and sometimes T2

In [55]:
RegSeason.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [56]:
RegSeasonSwitch = RegSeason[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore',  'WLoc',
       'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]
RegSeasonSwitch

,Season,DayNum,LTeamID,LScore,WTeamID,WScore,WLoc,NumOT,LFGM,LFGA,...,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF
0,2003,10,1328,62,1104,68,N,0,22,53,...,14,11,18,14,24,13,23,7,1,22
1,2003,10,1393,63,1272,70,N,0,24,67,...,20,10,19,15,28,16,13,4,4,18
2,2003,11,1437,61,1266,73,N,0,22,73,...,18,17,29,17,26,15,10,5,2,25
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1208,71,1400,77,N,0,24,62,...,14,11,13,17,22,12,14,4,4,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186657,2024,100,3427,70,3409,74,H,0,26,65,...,25,13,18,5,22,19,19,8,4,20
186658,2024,100,3161,64,3424,67,H,0,26,59,...,27,6,13,11,28,14,11,4,0,12
186659,2024,100,3296,47,3444,65,A,0,17,50,...,22,17,27,4,29,11,7,11,1,16
186660,2024,100,3347,51,3457,54,H,1,22,61,...,18,14,23,15,31,4,20,8,2,14


In [57]:
RegSeasonSwitch.loc[RegSeasonSwitch['WLoc'] == 'H', 'WLoc'] = 'A'
RegSeasonSwitch.loc[RegSeasonSwitch['WLoc'] == 'A', 'WLoc'] = 'H'
RegSeason.columns.values[6] = 'location'
RegSeasonSwitch.columns.values[6] = 'location'

RegSeason.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(RegSeason.columns)]
RegSeasonSwitch.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(RegSeasonSwitch.columns)]

In [59]:
RegSeason.head(15)

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
5,2003,11,1458,81,1186,55,H,0,26,57,...,11,12,17,6,22,8,19,4,3,25
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1186,75,1457,61,N,0,28,62,...,17,17,23,8,25,10,15,14,8,18
8,2003,12,1194,71,1156,66,N,0,28,58,...,18,12,27,13,26,13,25,8,2,18
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


In [60]:
RegSeasonSwitch.head(15)

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1328,62,1104,68,N,0,22,53,...,14,11,18,14,24,13,23,7,1,22
1,2003,10,1393,63,1272,70,N,0,24,67,...,20,10,19,15,28,16,13,4,4,18
2,2003,11,1437,61,1266,73,N,0,22,73,...,18,17,29,17,26,15,10,5,2,25
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1208,71,1400,77,N,0,24,62,...,14,11,13,17,22,12,14,4,4,20
5,2003,11,1186,55,1458,81,H,0,20,46,...,12,23,27,12,24,12,9,9,3,18
6,2003,12,1236,62,1161,80,H,0,19,41,...,8,32,39,13,18,14,17,11,1,25
7,2003,12,1457,61,1186,75,N,0,20,59,...,14,15,21,13,35,19,19,7,2,21
8,2003,12,1156,66,1194,71,N,0,24,52,...,11,10,18,9,22,9,17,9,2,23
9,2003,12,1296,56,1458,84,H,0,23,52,...,17,15,19,14,22,11,6,12,0,13


Looks like theyre still in order. I thought about just combining them and having 2 rows per game, but im thinking thats probably not a great idea since id be training on games that will also be in the test set, so im going to randomly select 50 percent from RegSeason where team 1 is the willer and the remaining indices will come from RegSeasonSwitch where team2 is the winner


In [61]:
RegSeason.shape

(186662, 34)

In [62]:
RegSeasonSwitch.shape

(186662, 34)

In [83]:
half = RegSeason.shape[0]//2

print(half)
np.random.seed(123)
half_indices = random.sample(list(RegSeason.index), k=half)

RegSeasonHalf = RegSeason.iloc[half_indices].sort_values(['Season', 'DayNum'])
print(RegSeasonHalf.head(20))

RegSeasonSwitchHalf = RegSeasonSwitch.drop(half_indices).sort_values(['Season', 'DayNum'])
RegSeasonSwitchHalf.head(20)

93331
    Season  DayNum  T1_TeamID  T1_Score  T2_TeamID  T2_Score location  NumOT  \
0     2003      10       1104        68       1328        62        N      0   
1     2003      10       1272        70       1393        63        N      0   
2     2003      11       1266        73       1437        61        N      0   
5     2003      11       1458        81       1186        55        H      0   
6     2003      12       1161        80       1236        62        H      0   
9     2003      12       1458        84       1296        56        H      0   
8     2003      12       1194        71       1156        66        N      0   
11    2003      13       1202        74       1106        73        N      0   
23    2003      14       1426        59       1106        47        N      0   
20    2003      14       1323        89       1237        45        H      0   
22    2003      14       1390        61       1131        57        H      0   
17    2003      14       1166     

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1208,71,1400,77,N,0,24,62,...,14,11,13,17,22,12,14,4,4,20
7,2003,12,1457,61,1186,75,N,0,20,59,...,14,15,21,13,35,19,19,7,2,21
10,2003,13,1426,50,1166,106,H,0,17,52,...,25,9,13,15,29,21,11,10,6,16
12,2003,13,1135,65,1237,66,N,0,24,56,...,19,9,13,21,23,15,17,12,3,17
13,2003,13,1125,48,1323,76,H,0,18,64,...,23,16,23,8,35,18,13,14,19,13
14,2003,14,1135,77,1125,83,N,1,28,60,...,31,12,15,15,18,22,16,18,5,21
15,2003,14,1236,71,1156,78,N,0,23,50,...,18,14,24,8,25,18,15,2,6,18
16,2003,14,1194,56,1161,81,H,0,21,65,...,12,32,43,16,32,13,24,5,3,19
18,2003,14,1129,59,1183,73,H,0,21,51,...,19,6,8,8,21,17,15,10,1,19


In [89]:
RegSeasonGames = pd.concat([RegSeasonHalf, RegSeasonSwitchHalf]).sort_index()
RegSeasonGames.head(25)

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1457,50,1296,56,N,0,18,49,...,9,17,31,6,19,11,12,14,2,18
4,2003,11,1208,71,1400,77,N,0,24,62,...,14,11,13,17,22,12,14,4,4,20
5,2003,11,1458,81,1186,55,H,0,26,57,...,11,12,17,6,22,8,19,4,3,25
6,2003,12,1161,80,1236,62,H,0,23,55,...,15,20,28,9,21,11,30,10,4,28
7,2003,12,1457,61,1186,75,N,0,20,59,...,14,15,21,13,35,19,19,7,2,21
8,2003,12,1194,71,1156,66,N,0,28,58,...,18,12,27,13,26,13,25,8,2,18
9,2003,12,1458,84,1296,56,H,0,32,67,...,14,7,12,9,23,10,18,1,3,18


In [85]:
RegSeasonGames.shape

(186662, 34)

## COmputing Season Stats By Team

Now to compute season averages. I Want to group by Team 1. I guess i can use a df with every game twice since theyre just averages


In [96]:
RegSeasonDouble = pd.concat([RegSeason, RegSeasonSwitch])
RegSeasonDouble.shape

(373324, 34)

In [99]:
RegSeasonDouble.columns

Index(['Season', 'DayNum', 'TeamID', 'Score', 'Opp_TeamID', 'Opp_Score',
       'location', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3',
       'Opp_FGA3', 'Opp_FTM', 'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast',
       'Opp_TO', 'Opp_Stl', 'Opp_Blk', 'Opp_PF'],
      dtype='object')

In [98]:
RegSeasonDouble.columns = [x.replace('T1_','').replace('T2_','Opp_') for x in list(RegSeasonDouble.columns)]
RegSeasonDouble.columns

### Look Into Calculating some more advanced stats

Below is just very basic averages

In [101]:
Stats = RegSeasonDouble.groupby(['TeamID', 'Season'])[['Score', 'Opp_Score',
       'location', 'NumOT', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR',
       'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'Opp_FGM', 'Opp_FGA', 'Opp_FGM3',
       'Opp_FGA3', 'Opp_FTM', 'Opp_FTA', 'Opp_OR', 'Opp_DR', 'Opp_Ast',
       'Opp_TO', 'Opp_Stl', 'Opp_Blk', 'Opp_PF']].mean().reset_index()

In [102]:
Stats

,TeamID,Season,Score,Opp_Score,NumOT,FGM,FGA,FGM3,FGA3,FTM,...,Opp_FGA3,Opp_FTM,Opp_FTA,Opp_OR,Opp_DR,Opp_Ast,Opp_TO,Opp_Stl,Opp_Blk,Opp_PF
0,1101,2014,63.142857,78.619048,0.142857,20.333333,50.142857,6.666667,17.857143,15.809524,...,16.190476,18.333333,25.809524,10.380952,24.095238,15.571429,12.142857,7.000000,5.000000,18.666667
1,1101,2015,61.000000,71.857143,0.000000,21.428571,52.928571,7.250000,19.178571,10.892857,...,16.714286,15.607143,22.714286,10.035714,25.892857,12.928571,13.464286,5.857143,4.250000,16.535714
2,1101,2016,69.851852,76.259259,0.074074,23.814815,53.925926,6.851852,18.851852,15.370370,...,15.148148,17.703704,24.962963,8.592593,25.222222,12.851852,13.444444,5.111111,3.296296,19.666667
3,1101,2017,67.880000,72.640000,0.160000,24.440000,53.320000,7.080000,19.080000,11.920000,...,19.600000,15.880000,23.800000,10.640000,25.040000,13.600000,14.800000,6.040000,3.160000,17.240000
4,1101,2018,70.666667,72.814815,0.037037,25.518519,56.481481,6.481481,19.962963,13.148148,...,19.740741,16.629630,23.444444,9.925926,25.037037,12.037037,15.481481,6.518519,2.925926,17.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3476,2023,59.178571,65.428571,0.035714,21.571429,55.821429,6.178571,20.250000,9.857143,...,15.821429,10.071429,14.464286,8.785714,22.964286,11.785714,11.214286,7.892857,3.214286,16.142857
12853,3476,2024,51.391304,69.086957,0.000000,18.304348,55.565217,5.478261,20.173913,9.304348,...,18.086957,9.434783,14.347826,11.434783,24.739130,14.217391,14.043478,11.565217,3.434783,16.608696
12854,3477,2023,64.625000,69.625000,0.031250,22.937500,58.218750,3.656250,14.250000,15.093750,...,19.781250,10.281250,14.625000,9.593750,23.406250,15.437500,15.781250,9.000000,4.062500,18.562500
12855,3477,2024,68.000000,75.600000,0.050000,24.500000,66.350000,7.000000,21.600000,12.000000,...,19.450000,13.900000,19.250000,11.900000,31.000000,13.850000,13.400000,6.650000,3.800000,16.300000


Games Data Doesnt need the stats anymore

In [103]:
RegSeasonGames.columns

Index(['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID', 'T2_Score',
       'location', 'NumOT', 'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM',
       'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk',
       'T1_PF', 'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA',
       'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF'],
      dtype='object')

In [ ]:
RegSeasonGame = RegSeasonGames[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID', 'T2_Score',
       'location']]
RegSeasonGame

Eventually ill Have to merge the RegSeasonGame with that teams season stats, I want to make some more advanced stats frist though
When I merge, Remember column names will ahve to be the same whcih will probably involve making 2 identical datasets out of the Stats Df, only differnce being the column names where 'TeamId' will become 'T1_TeamID' in one and 'T2_TeamID' in the other. All other columns will change similarly. For instance, 'Opp_FGM" becomes 'T1_Opp_FGM'

Also Ill have to do the same for tourney games and conference tourney games

I can already merge GameRecords and Stats

In [106]:
TeamRecords.shape

(12812, 7)

In [107]:
Stats.shape

(12857, 31)

In [110]:
pd.merge(Stats, TeamRecords, 'left')

,TeamID,Season,Score,Opp_Score,NumOT,FGM,FGA,FGM3,FGA3,FTM,...,Opp_Ast,Opp_TO,Opp_Stl,Opp_Blk,Opp_PF,Wins,Losses,OpponentsBeaten,OpponentsLost,Win_PCT
0,1101,2014,63.142857,78.619048,0.142857,20.333333,50.142857,6.666667,17.857143,15.809524,...,15.571429,12.142857,7.000000,5.000000,18.666667,2.0,19.0,"[1146, 1223]","[1182, 1382, 1268, 1234, 1395, 1462, 1406, 130...",0.095238
1,1101,2015,61.000000,71.857143,0.000000,21.428571,52.928571,7.250000,19.178571,10.892857,...,12.928571,13.464286,5.857143,4.250000,16.535714,7.0,21.0,"[1170, 1354, 1115, 1146, 1311, 1322, 1223]","[1409, 1182, 1415, 1222, 1260, 1129, 1213, 137...",0.250000
2,1101,2016,69.851852,76.259259,0.074074,23.814815,53.925926,6.851852,18.851852,15.370370,...,12.851852,13.444444,5.111111,3.296296,19.666667,9.0,18.0,"[1197, 1146, 1322, 1270, 1249, 1223, 1249, 136...","[1339, 1161, 1324, 1306, 1304, 1415, 1395, 117...",0.333333
3,1101,2017,67.880000,72.640000,0.160000,24.440000,53.320000,7.080000,19.080000,11.920000,...,13.600000,14.800000,6.040000,3.160000,17.240000,9.0,16.0,"[1306, 1149, 1372, 1249, 1309, 1358, 1322, 127...","[1149, 1328, 1307, 1170, 1362, 1309, 1368, 114...",0.360000
4,1101,2018,70.666667,72.814815,0.037037,25.518519,56.481481,6.481481,19.962963,13.148148,...,12.037037,15.481481,6.518519,2.925926,17.666667,12.0,15.0,"[1144, 1132, 1102, 1402, 1358, 1270, 1223, 132...","[1117, 1252, 1367, 1252, 1403, 1309, 1368, 137...",0.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12852,3476,2023,59.178571,65.428571,0.035714,21.571429,55.821429,6.178571,20.250000,9.857143,...,11.785714,11.214286,7.892857,3.214286,16.142857,8.0,20.0,"[3216, 3306, 3136, 3148, 3254, 3383, 3148, 3447]","[3250, 3344, 3138, 3193, 3436, 3324, 3221, 310...",0.285714
12853,3476,2024,51.391304,69.086957,0.000000,18.304348,55.565217,5.478261,20.173913,9.304348,...,14.217391,14.043478,11.565217,3.434783,16.608696,3.0,20.0,"[3192, 3447, 3384]","[3318, 3177, 3250, 3136, 3138, 3310, 3469, 322...",0.130435
12854,3477,2023,64.625000,69.625000,0.031250,22.937500,58.218750,3.656250,14.250000,15.093750,...,15.437500,15.781250,9.000000,4.062500,18.562500,13.0,19.0,"[3225, 3230, 3311, 3270, 3223, 3249, 3309, 322...","[3426, 3222, 3374, 3307, 3381, 3469, 3280, 313...",0.406250
12855,3477,2024,68.000000,75.600000,0.050000,24.500000,66.350000,7.000000,21.600000,12.000000,...,13.850000,13.400000,6.650000,3.800000,16.300000,9.0,11.0,"[3469, 3225, 3381, 3270, 3223, 3249, 3311, 327...","[3307, 3403, 3450, 3222, 3401, 3309, 3230, 336...",0.450000
